In [1]:
import pandas as pd

import requests

import json

import regex as re

pd.set_option('display.max_columns' , None)

1. Extracción de Datos a Nivel Nacional:

- Extraer datos de generación de energía renovable y no renovable a nivel nacional desde el 1 de enero de 2011 hasta el 31 de diciembre de 2022. La información debe ser recopilada a nivel mensual.
- Transformar los datos para incluir una columna nueva que identifique si la energía es renovable o no renovable.
- Almacenar los datos en un DataFrame con la siguiente estructura:

In [14]:
datos_acumulados = {'included' : []}

for anio in range (0 , 11 , 2):
    
    url_datos_mensuales = f'https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={2011+anio}-01-01T00:00&end_date={2012+anio}-12-31T23:59&time_trunc=month'
    datos_respuesta = requests.get(url_datos_mensuales)
    
    if datos_respuesta.status_code == 200:
        datos_nuevos = datos_respuesta.json()
        datos_acumulados['included'].append(datos_nuevos['included'])
        
    else:
        print(f"El año {2011+anio} - {2012+anio} ha tenido un problema")
    

In [70]:
def limpieza_datos(json):
    
    nuevo_diccionario = {'tipo_energia' : [] , 'tipo_generacion' : [] , 'fecha' : [] , 'valor' : []}
    tipo_energia = []
    tipo_generacion = [] 
    fecha = []
    valor =  []
    
    for anio in json['included']:
        for elemento in anio:
            tipo_energia.extend([elemento['type']] * len(elemento['attributes']['values']))
            tipo_generacion.extend([elemento['attributes']['type']] * len(elemento['attributes']['values']))
            
            for dato in elemento['attributes']['values']:
                fecha.append(dato['datetime'])
                valor.append(dato['value'])
    
    nuevo_diccionario['tipo_energia'] = tipo_energia
    nuevo_diccionario['tipo_generacion'] = tipo_generacion
    nuevo_diccionario['fecha'] = fecha
    nuevo_diccionario['valor'] = valor
    
    return nuevo_diccionario
            
    

In [74]:
diccionario_mensual = limpieza_datos(datos_acumulados)

In [75]:
pd.DataFrame(diccionario_mensual)

,tipo_energia,tipo_generacion,fecha,valor
0,Hidráulica,Renovable,2011-01-01T00:00:00.000+01:00,5.119512e+06
1,Hidráulica,Renovable,2011-02-01T00:00:00.000+01:00,2.998051e+06
2,Hidráulica,Renovable,2011-03-01T00:00:00.000+01:00,3.910363e+06
3,Hidráulica,Renovable,2011-04-01T00:00:00.000+02:00,3.537938e+06
4,Hidráulica,Renovable,2011-05-01T00:00:00.000+02:00,2.797500e+06
...,...,...,...,...
2482,Generación total,Generación total,2022-08-01T00:00:00.000+02:00,2.528589e+07
2483,Generación total,Generación total,2022-09-01T00:00:00.000+02:00,2.366621e+07
2484,Generación total,Generación total,2022-10-01T00:00:00.000+02:00,2.195664e+07
2485,Generación total,Generación total,2022-11-01T00:00:00.000+01:00,2.152262e+07
